### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

--2025-10-31 15:01:42--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6026:18::a27d:4612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/0mulrothty5o8i8ud9gz2/arxivData.json.tar.gz?rlkey=n759u5qx2xpxxglmrl390vwvk&dl=1 [following]
--2025-10-31 15:01:43--  https://www.dropbox.com/scl/fi/0mulrothty5o8i8ud9gz2/arxivData.json.tar.gz?rlkey=n759u5qx2xpxxglmrl390vwvk&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc62ead9195c82148c18472decb1.dl.dropboxusercontent.com/cd/0/inline/C0Q2eR2BH9M8sGyAu2eCLEVxF_d8aZokliW-ftMTfrUAyG_bryiPZ5ZxTRf6qCqesVWV88ub7cGU6-ajkqyPdlaUUtfep-LGehjpLuwCJckuA-cDHbYOZUhFeG5BzomG6Iw/file?dl=1# [following]
--2025-10-31 15:01:44--  https://uc62ead9195c82148c18472decb1.dl.dropbox

,author,day,id,link,month,summary,tag,title,year
17519,"[{'name': 'Diego Moussallem'}, {'name': 'Matth...",26,1711.09476v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,A large number of machine translation approach...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Machine Translation Using Semantic Web Technol...,2017
16282,"[{'name': 'Sophie J. Lee'}, {'name': 'Howard L...",14,1611.04837v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,"Extracting the ""correct"" location information ...","[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Lost in Space: Geolocation in Event Data,2016
34322,"[{'name': 'Antti Suni'}, {'name': 'Daniel Aalt...",7,1510.01949v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",10,Prominences and boundaries are the essential c...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Hierarchical Representation of Prosody for Sta...,2015
2429,"[{'name': 'Hei Chan'}, {'name': 'Adnan Darwich...",19,1212.2470v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",10,Bayesian network classifiers are used in many ...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Reasoning about Bayesian Network Classifiers,2012
33276,[{'name': 'Christoph Schommer'}],9,0805.1296v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",5,"In this paper, we informally introduce dynamic...","[{'term': 'cs.NE', 'scheme': 'http://arxiv.org...",A Simple Dynamic Mind-map Framework To Discove...,2008


In [4]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'].replace("\n", ' '), axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [5]:
# Task: convert lines (in-place) into strings of space-separated tokens. Import & use WordPunctTokenizer

from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

lines = [
    ' '.join(tokenizer.tokenize(line.lower().strip()))
    for line in lines
    if isinstance(line, str) and line.strip()
]

In [6]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model (1point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words.

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [7]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens:
# - `UNK` represents absent tokens,
# - `EOS` is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases:
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)

    for line in tqdm(lines):
        tokens = line.split()
        tokens.append(EOS)  # добавляем EOS в конец

        for i in range(len(tokens)):
            # берём последние (n - 1) токенов перед текущим
            prefix = tokens[max(0, i - (n - 1)):i]
            # если их меньше чем (n - 1), добавляем UNK слева
            if len(prefix) < n - 1:
                prefix = [UNK] * (n - 1 - len(prefix)) + prefix

            prefix = tuple(prefix)
            word = tokens[i]

            counts[prefix][word] += 1

    return counts


In [8]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 44648.75it/s]


Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [9]:
class NGramLanguageModel:
    def __init__(self, lines, n):
        """
        Train a simple count-based language model:
        compute probabilities P(w_t | prefix) given ngram counts

        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n

        counts = count_ngrams(lines, self.n)

        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)

        # populate self.probs with actual probabilities
        for prefix, counter in counts.items():
            total = sum(counter.values())
            for token, cnt in counter.items():
                self.probs[prefix][token] = cnt / total  # нормализация по сумме

    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]

    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [10]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 28068.69it/s]


Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [11]:
lm = NGramLanguageModel(lines, n=3)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 41000/41000 [00:07<00:00, 5628.01it/s]


The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [12]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    probs_dict = lm.get_possible_next_tokens(prefix)

    if not probs_dict:  # если модель не знает таких токенов — вернём UNK
        return UNK

    tokens = np.array(list(probs_dict.keys()))
    probs = np.array(list(probs_dict.values()), dtype=np.float64)

    if temperature == 0:  # детерминированный выбор (argmax)
        return tokens[np.argmax(probs)]

    # применяем "смягчение" распределения температурой
    scaled_probs = probs ** (1.0 / temperature)
    scaled_probs /= scaled_probs.sum()  # нормализуем

    # выбираем токен случайно, пропорционально вероятностям
    next_token = np.random.choice(tokens, p=scaled_probs)
    return next_token

In [13]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [14]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

artificial intelligence ( ai ) capabilities in planning with pixels of an " art " flat " minima , as well as with cox ' s formal defaults , we present analyses of the baldwin effect , complex , structured - prediction task , which poses an increased thrust in exploring temporal coherence of nonlocal patches . this process suffers from combinatorial design theory are applicable to a canonical model , which can mimic the motion information as a result , methods , and the planet , and hypergraphs ; community - based acoustic auto - localizing the visual vocabulary acquisition


In [15]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

bridging the gap between the two - stage methods have been widely used for the segmentation of the most important features by the lack of competitors per explosion , under the assumption of the data . _EOS_


__More in the homework:__ nucleus sampling, top-k sampling, beam search(not for the faint of heart).

### Evaluating language models: perplexity (1point)

Perplexity is a measure of how well your model approximates the true probability distribution behind the data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of $1/N$, where $N$ is __total length (in tokens) of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [16]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above

    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)

    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    total_log_prob = 0.0
    total_tokens = 0

    for line in lines:
        tokens = line.strip().split()
        tokens.append(EOS)  # обязательно добавляем конец предложения

        # перебираем токены, включая первый (где префикс пустой)
        for i in range(len(tokens)):
            prefix = tokens[max(0, i - (lm.n - 1)):i]

            # если длина префикса меньше (n-1) → добавляем UNK слева
            if len(prefix) < lm.n - 1:
                prefix = [UNK] * ((lm.n - 1) - len(prefix)) + prefix

            prefix_str = ' '.join(prefix)
            token = tokens[i]

            # получаем вероятность токена
            prob = lm.get_next_token_prob(prefix_str, token)

            # избегаем log(0)
            logprob = np.log(prob) if prob > 0 else min_logprob

            total_log_prob += logprob
            total_tokens += 1

    # log perplexity = - (1/N) * sum(log P)
    avg_log_prob = total_log_prob / total_tokens
    ppl = np.exp(-avg_log_prob)

    return ppl

In [17]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be non-negative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 38642.93it/s]

Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [18]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 30750/30750 [00:01<00:00, 20183.67it/s]


N = 1, Perplexity = 1832.23136


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 30750/30750 [00:02<00:00, 12307.92it/s]


N = 2, Perplexity = 85653987.28774


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 30750/30750 [00:05<00:00, 5374.65it/s]


N = 3, Perplexity = 61999196259043346743296.00000


In [19]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [20]:
class LaplaceLanguageModel(NGramLanguageModel):
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}

    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))


**Disclaimer**: the implementation above assumes all words unknown within a given context to be equally likely, *as well as the words outside of vocabulary*. Therefore, its' perplexity will be lower than it should when encountering such words. Therefore, comparing it with a model with fewer unknown words will not be fair. When implementing your own smoothing, you may handle this by adding a virtual `UNK` token of non-zero probability. Technically, this will result in a model where probabilities do not add up to $1$, but it is close enough for a practice excercise.

In [21]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 51438.61it/s]


In [22]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 30750/30750 [00:01<00:00, 19906.41it/s]


N = 1, Perplexity = 1832.66878


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 30750/30750 [00:02<00:00, 12366.59it/s]


N = 2, Perplexity = 470.48021


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 30750/30750 [00:05<00:00, 5508.69it/s]


N = 3, Perplexity = 3679.44765


In [23]:
# optional: try to sample tokens from such a model

### Kneser-Ney smoothing (2 points)

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement `KneserNeyLanguageModel` class,
- test it on 1-3 gram language models
- find optimal (within reason) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [ ]:
class KneserNeyLanguageModel(NGramLanguageModel):
    """A simple Kneser-Ney language model template"""
    def __init__(self, lines, n, delta=0.75):
        self.n = n
        self.delta = delta

        self.tokens = [line.split() for line in lines]

        self.vocab = set()
        for tokens in self.tokens:
            self.vocab.update(tokens)
        self.vocab = sorted(list(self.vocab))

        self.counts = Counter()
        self.context_counts = Counter()
        for tokens in self.tokens:
            tokens = ['<s>'] * (self.n - 1) + tokens + ['</s>']
            for i in range(len(tokens) - self.n + 1):
                ngram = tuple(tokens[i:i+self.n])
                prefix = ngram[:-1]
                self.counts[ngram] += 1
                self.context_counts[prefix] += 1

    def get_possible_next_tokens(self, prefix):
        if isinstance(prefix, str):
            prefix = (prefix,)
        if len(prefix) < self.n - 1:
            prefix = ('<s>',) * (self.n - 1 - len(prefix)) + tuple(prefix)

        return [ngram[-1] for ngram in self.counts if ngram[:-1] == prefix]

    def get_next_token_prob(self, prefix, next_token):
        if isinstance(prefix, str):
            prefix = (prefix,)
        if len(prefix) < self.n - 1:
            prefix = ('<s>',) * (self.n - 1 - len(prefix)) + tuple(prefix)

        ngram = prefix + (next_token,)
        c_ngram = self.counts.get(ngram, 0)
        c_prefix = self.context_counts.get(prefix, 0)

        if c_prefix == 0:
            return 1 / len(self.vocab)
    
        p_raw = max(c_ngram - self.delta, 0) / c_prefix

        all_probs = [
            max(self.counts.get(prefix + (w,), 0) - self.delta, 0)
            for w in self.vocab
        ]
        total = sum(all_probs)
        if total == 0:
            return 1 / len(self.vocab)

        return (max(c_ngram - self.delta, 0)) / total

In [ ]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [ ]:
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines, n=n, delta=0.5)  # или 0.5, 1.0 — можно поэкспериментировать
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))